# OTC Market Stock Trading Strategy
## Alpaca Platform - Automated Trading Bot

**Strategy Overview:**
- Screen OTC stocks priced between $0.3 - $2.00
- Identify stocks with increasing volume over 3 consecutive days
- Select top 3 candidates daily
- Buy $1000 worth daily for 5 days at market price
- Auto-sell when price reaches 50% above average cost


In [ ]:
# Install required packages
!pip install alpaca-trade-api pandas numpy -q


In [ ]:
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import json
from typing import List, Dict, Optional


In [ ]:
# Alpaca API Configuration
BASE_URL = 'https://paper-api.alpaca.markets/v2'
API_KEY = 'PKTQBTL5H2JAHP2AOGKYUNMXST'
API_SECRET = 'GjwMgV9MUu34SNQAcYsf4s5ctipjfTQqUvuuWdzbn6G1'

# Strategy Parameters
MIN_PRICE = 0.3
MAX_PRICE = 2.0
DAILY_PURCHASE_AMOUNT = 1000  # dollars
BUY_DAYS = 5  # Buy for 5 consecutive days
VOLUME_ACTIVITY_DAYS = 3  # Volume should be active for 3 days
PROFIT_TARGET_PCT = 50  # Sell when 50% profit
MAX_STOCKS_PER_DAY = 3  # Select top 3 stocks daily

# Initialize Alpaca API
api = tradeapi.REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')

print("Alpaca API initialized successfully!")
print(f"Account Status: {api.get_account().status}")


## Core Functions


In [ ]:
def get_otc_stocks() -> List[Dict]:
    """
    Get list of OTC stocks from Alpaca.
    Note: Alpaca may have limited OTC symbols. This function attempts to get available symbols.
    """
    try:
        # Get all assets (stocks)
        assets = api.list_assets(status='active', asset_class='us_equity')
        
        # Filter OTC stocks (typically have 4-5 characters and may include 'OTC' or specific patterns)
        # Note: Alpaca's OTC coverage may vary. You may need to manually maintain a list.
        otc_stocks = []
        
        for asset in assets:
            # OTC stocks are typically not on major exchanges
            if asset.tradable and hasattr(asset, 'exchange') and asset.exchange in ['OTC', 'OTCQB', 'OTCQX', 'OTCMKTS']:
                otc_stocks.append({
                    'symbol': asset.symbol,
                    'name': asset.name,
                    'exchange': asset.exchange
                })
        
        print(f"Found {len(otc_stocks)} OTC stocks")
        return otc_stocks
    except Exception as e:
        print(f"Error fetching OTC stocks: {e}")
        # Fallback: Return empty list (user should provide symbols)
        print("Note: Please provide a list of OTC symbols if Alpaca doesn't return any.")
        return []


In [ ]:
def check_volume_increase(symbol: str, days: int = VOLUME_ACTIVITY_DAYS) -> bool:
    """
    Check if a stock has increasing volume over the specified number of days.
    Returns True if volume has been consistently active/increasing for the period.
    """
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days + 5)  # Get extra days for comparison
        
        # Get historical bars
        bars = api.get_bars(
            symbol,
            TimeFrame.Day,
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d'),
            limit=1000
        ).df
        
        if len(bars) < days + 2:
            return False
        
        # Get recent volumes (last 'days' days)
        recent_volumes = bars['volume'].tail(days).values
        # Get previous volumes for comparison
        previous_volumes = bars['volume'].tail(days + 3).head(3).values if len(bars) >= days + 3 else []
        
        # Check if recent volumes are significantly higher than previous average
        if len(previous_volumes) > 0:
            avg_previous_volume = np.mean(previous_volumes)
            avg_recent_volume = np.mean(recent_volumes)
            
            # Volume should be at least 50% higher than previous average
            if avg_recent_volume > avg_previous_volume * 1.5:
                # Also check for consistent increase trend
                if all(recent_volumes[i] >= recent_volumes[i-1] * 0.8 for i in range(1, len(recent_volumes))):
                    return True
        
        return False
        
    except Exception as e:
        print(f"Error checking volume for {symbol}: {e}")
        return False


In [ ]:
def get_stock_price(symbol: str) -> Optional[float]:
    """
    Get current price of a stock.
    """
    try:
        # Get latest bar
        bars = api.get_bars(symbol, TimeFrame.Day, limit=1).df
        if len(bars) > 0:
            return float(bars['close'].iloc[-1])
        
        # Alternative: Get latest quote
        quote = api.get_latest_quote(symbol)
        if quote:
            return float((quote.bp + quote.ap) / 2)  # Mid price
        
        return None
    except Exception as e:
        print(f"Error getting price for {symbol}: {e}")
        return None


In [ ]:
def screen_stocks(symbols: List[str]) -> List[Dict]:
    """
    Screen stocks based on price range and volume activity.
    Returns list of qualified stocks with their metrics.
    """
    qualified_stocks = []
    
    print(f"\nScreening {len(symbols)} stocks...")
    
    for symbol in symbols:
        try:
            # Check price range
            price = get_stock_price(symbol)
            if price is None or price < MIN_PRICE or price > MAX_PRICE:
                continue
            
            # Check volume increase
            if not check_volume_increase(symbol, VOLUME_ACTIVITY_DAYS):
                continue
            
            # Get volume data for ranking
            bars = api.get_bars(symbol, TimeFrame.Day, limit=VOLUME_ACTIVITY_DAYS).df
            avg_volume = bars['volume'].mean() if len(bars) > 0 else 0
            
            qualified_stocks.append({
                'symbol': symbol,
                'price': price,
                'avg_volume': avg_volume,
                'volume_change_pct': ((bars['volume'].iloc[-1] / bars['volume'].iloc[0]) - 1) * 100 if len(bars) >= 2 else 0
            })
            
            print(f"✓ Qualified: {symbol} @ ${price:.2f}, Avg Volume: {avg_volume:,.0f}")
            
        except Exception as e:
            print(f"Error screening {symbol}: {e}")
            continue
    
    # Sort by volume change percentage (most active first)
    qualified_stocks.sort(key=lambda x: x['volume_change_pct'], reverse=True)
    
    return qualified_stocks[:MAX_STOCKS_PER_DAY]


In [ ]:
def get_positions() -> Dict[str, Dict]:
    """
    Get all current positions from Alpaca account.
    Returns a dictionary with symbol as key and position info as value.
    """
    try:
        positions = api.list_positions()
        positions_dict = {}
        
        for pos in positions:
            positions_dict[pos.symbol] = {
                'qty': float(pos.qty),
                'avg_entry_price': float(pos.avg_entry_price),
                'current_price': float(pos.current_price),
                'market_value': float(pos.market_value),
                'unrealized_pl': float(pos.unrealized_pl),
                'unrealized_plpc': float(pos.unrealized_plpc) * 100  # as percentage
            }
        
        return positions_dict
    except Exception as e:
        print(f"Error getting positions: {e}")
        return {}


In [ ]:
# Global tracking for buy schedules (in production, use a database)
# Format: {symbol: {'start_date': date, 'days_bought': int, 'total_cost': float, 'total_shares': float}}
buy_schedules = {}

def load_buy_schedules() -> Dict:
    """Load buy schedules from file (for persistence across runs)"""
    try:
        with open('buy_schedules.json', 'r') as f:
            data = json.load(f)
            # Convert date strings back to datetime
            for symbol, schedule in data.items():
                schedule['start_date'] = datetime.fromisoformat(schedule['start_date'])
            return data
    except FileNotFoundError:
        return {}
    except Exception as e:
        print(f"Error loading buy schedules: {e}")
        return {}

def save_buy_schedules():
    """Save buy schedules to file"""
    try:
        data = {}
        for symbol, schedule in buy_schedules.items():
            data[symbol] = schedule.copy()
            data[symbol]['start_date'] = schedule['start_date'].isoformat()
        
        with open('buy_schedules.json', 'w') as f:
            json.dump(data, f, indent=2)
    except Exception as e:
        print(f"Error saving buy schedules: {e}")

# Load existing schedules
try:
    buy_schedules = load_buy_schedules()
except:
    buy_schedules = {}


In [ ]:
def execute_buy(symbol: str) -> bool:
    """
    Execute a market buy order for the specified amount.
    """
    try:
        current_price = get_stock_price(symbol)
        if current_price is None:
            print(f"Cannot get price for {symbol}, skipping buy")
            return False
        
        # Calculate number of shares to buy (round down to whole shares)
        shares_to_buy = int(DAILY_PURCHASE_AMOUNT / current_price)
        
        if shares_to_buy < 1:
            print(f"Not enough funds to buy at least 1 share of {symbol} at ${current_price}")
            return False
        
        # Place market order
        order = api.submit_order(
            symbol=symbol,
            qty=shares_to_buy,
            side='buy',
            type='market',
            time_in_force='day'
        )
        
        print(f"✓ Buy order submitted for {symbol}: {shares_to_buy} shares @ ~${current_price:.2f}")
        
        # Wait for order to fill
        time.sleep(2)
        
        # Update buy schedule
        if symbol not in buy_schedules:
            buy_schedules[symbol] = {
                'start_date': datetime.now(),
                'days_bought': 0,
                'total_cost': 0.0,
                'total_shares': 0.0
            }
        
        # Calculate actual cost (approximate)
        actual_cost = shares_to_buy * current_price
        buy_schedules[symbol]['days_bought'] += 1
        buy_schedules[symbol]['total_cost'] += actual_cost
        buy_schedules[symbol]['total_shares'] += shares_to_buy
        
        save_buy_schedules()
        
        return True
        
    except Exception as e:
        print(f"Error executing buy for {symbol}: {e}")
        return False


In [ ]:
def execute_sell(symbol: str, reason: str = "Profit target reached") -> bool:
    """
    Execute a market sell order for all shares of a symbol.
    """
    try:
        positions = get_positions()
        
        if symbol not in positions:
            print(f"No position found for {symbol}")
            return False
        
        qty = positions[symbol]['qty']
        current_price = positions[symbol]['current_price']
        
        if qty <= 0:
            print(f"Invalid quantity for {symbol}: {qty}")
            return False
        
        # Place market sell order
        order = api.submit_order(
            symbol=symbol,
            qty=int(qty),
            side='sell',
            type='market',
            time_in_force='day'
        )
        
        print(f"✓ Sell order submitted for {symbol}: {int(qty)} shares @ ~${current_price:.2f} - {reason}")
        
        # Remove from buy schedule tracking
        if symbol in buy_schedules:
            del buy_schedules[symbol]
            save_buy_schedules()
        
        return True
        
    except Exception as e:
        print(f"Error executing sell for {symbol}: {e}")
        return False


In [ ]:
def check_profit_targets():
    """
    Check all positions and sell if they reach profit target (50% above average cost).
    """
    positions = get_positions()
    
    for symbol, pos_info in positions.items():
        try:
            avg_entry = pos_info['avg_entry_price']
            current_price = pos_info['current_price']
            profit_pct = pos_info['unrealized_plpc']
            
            # Check if profit target reached (50% above average cost)
            if profit_pct >= PROFIT_TARGET_PCT:
                print(f"\n🎯 Profit target reached for {symbol}!")
                print(f"   Avg Entry: ${avg_entry:.2f}, Current: ${current_price:.2f}, Profit: {profit_pct:.2f}%")
                execute_sell(symbol, f"Profit target reached ({profit_pct:.2f}%)")
        except Exception as e:
            print(f"Error checking profit for {symbol}: {e}")


In [ ]:
def process_daily_buys():
    """
    Process daily buy orders for stocks in the buy schedule.
    """
    today = datetime.now().date()
    
    # Check existing buy schedules
    symbols_to_continue = []
    
    for symbol, schedule in list(buy_schedules.items()):
        start_date = schedule['start_date'].date() if isinstance(schedule['start_date'], datetime) else datetime.fromisoformat(schedule['start_date']).date()
        days_elapsed = (today - start_date).days
        days_bought = schedule['days_bought']
        
        # If we haven't bought today and still within 5-day window
        if days_bought < BUY_DAYS and days_elapsed < BUY_DAYS + 2:
            # Only buy if it's a new day (to prevent multiple buys on same day)
            if days_bought == 0 or days_elapsed >= days_bought:
                symbols_to_continue.append(symbol)
    
    # Execute buys for stocks in schedule
    for symbol in symbols_to_continue:
        execute_buy(symbol)
    
    # Clean up completed schedules
    for symbol, schedule in list(buy_schedules.items()):
        if schedule['days_bought'] >= BUY_DAYS:
            print(f"✓ Completed 5-day buy schedule for {symbol}")
            # Don't delete, keep for tracking


## Main Trading Loop


In [ ]:
def daily_screen_and_trade(otc_symbols: List[str] = None):
    """
    Main function to screen stocks and execute trades.
    """
    print("\n" + "="*60)
    print(f"Daily OTC Stock Screening - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*60)
    
    # Step 1: Check existing positions for profit targets
    print("\n[Step 1] Checking existing positions for profit targets...")
    check_profit_targets()
    
    # Step 2: Process daily buys for stocks in schedule
    print("\n[Step 2] Processing daily buy orders...")
    process_daily_buys()
    
    # Step 3: Screen for new stock candidates
    print("\n[Step 3] Screening for new stock candidates...")
    
    # Get OTC symbols if not provided
    if otc_symbols is None:
        otc_stocks = get_otc_stocks()
        otc_symbols = [stock['symbol'] for stock in otc_stocks]
    
    if not otc_symbols:
        print("No OTC symbols available. Please provide a list of OTC symbols.")
        return
    
    # If we already have MAX_STOCKS_PER_DAY in buy schedule, skip screening
    active_symbols_count = sum(1 for s in buy_schedules.values() if s['days_bought'] < BUY_DAYS)
    
    if active_symbols_count >= MAX_STOCKS_PER_DAY:
        print(f"Already have {active_symbols_count} active stocks in buy schedule. Skipping new screening.")
        return
    
    # Screen stocks
    qualified_stocks = screen_stocks(otc_symbols)
    
    if not qualified_stocks:
        print("\n❌ No qualified stocks found today. No new trades.")
        return
    
    # Step 4: Select top candidates (avoid duplicates)
    print(f"\n[Step 4] Selecting up to {MAX_STOCKS_PER_DAY} stocks from {len(qualified_stocks)} candidates...")
    
    selected_count = 0
    for stock in qualified_stocks:
        if selected_count >= MAX_STOCKS_PER_DAY:
            break
        
        symbol = stock['symbol']
        
        # Skip if already in buy schedule
        if symbol in buy_schedules:
            print(f"Skipping {symbol} - already in buy schedule")
            continue
        
        # Skip if already have a position
        positions = get_positions()
        if symbol in positions:
            print(f"Skipping {symbol} - already have position")
            continue
        
        # Start buy schedule
        print(f"\n📈 Starting buy schedule for {symbol} @ ${stock['price']:.2f}")
        if execute_buy(symbol):
            selected_count += 1
    
    if selected_count == 0:
        print("\nNo new stocks selected today.")
    else:
        print(f"\n✓ Started buy schedules for {selected_count} new stock(s)")
    
    # Summary
    print("\n" + "="*60)
    print("Trading Summary:")
    print("="*60)
    
    positions = get_positions()
    if positions:
        print(f"\nCurrent Positions ({len(positions)}):")
        for symbol, pos in positions.items():
            print(f"  {symbol}: {pos['qty']:.0f} shares @ ${pos['avg_entry_price']:.2f} avg")
            print(f"    Current: ${pos['current_price']:.2f}, P/L: {pos['unrealized_plpc']:.2f}%")
    else:
        print("\nNo current positions.")
    
    if buy_schedules:
        print(f"\nActive Buy Schedules ({len(buy_schedules)}):")
        for symbol, schedule in buy_schedules.items():
            avg_cost = schedule['total_cost'] / schedule['total_shares'] if schedule['total_shares'] > 0 else 0
            print(f"  {symbol}: {schedule['days_bought']}/{BUY_DAYS} days, ")
            print(f"    {schedule['total_shares']:.0f} shares @ ${avg_cost:.2f} avg")
    
    print("\n" + "="*60)


## Usage

### Run the Daily Screening and Trading


In [ ]:
# Run the daily screening and trading
# Option 1: Auto-detect OTC stocks (may be limited on Alpaca)
daily_screen_and_trade()

# Option 2: Provide a specific list of OTC symbols to screen
# daily_screen_and_trade(['OTC_SYMBOL1', 'OTC_SYMBOL2', 'OTC_SYMBOL3'])


## Monitoring & Utility Functions


In [ ]:
# View account information
account = api.get_account()
print(f"Account Status: {account.status}")
print(f"Buying Power: ${float(account.buying_power):,.2f}")
print(f"Cash: ${float(account.cash):,.2f}")
print(f"Portfolio Value: ${float(account.portfolio_value):,.2f}")
print(f"Equity: ${float(account.equity):,.2f}")


In [ ]:
# View all current positions
positions = get_positions()
if positions:
    print(f"\nCurrent Positions ({len(positions)}):")
    for symbol, pos in positions.items():
        print(f"\n{symbol}:")
        print(f"  Quantity: {pos['qty']:.0f} shares")
        print(f"  Avg Entry Price: ${pos['avg_entry_price']:.2f}")
        print(f"  Current Price: ${pos['current_price']:.2f}")
        print(f"  Market Value: ${pos['market_value']:,.2f}")
        print(f"  Unrealized P/L: ${pos['unrealized_pl']:,.2f} ({pos['unrealized_plpc']:.2f}%)")
else:
    print("No current positions.")


In [ ]:
# View active buy schedules
if buy_schedules:
    print(f"\nActive Buy Schedules ({len(buy_schedules)}):")
    for symbol, schedule in buy_schedules.items():
        avg_cost = schedule['total_cost'] / schedule['total_shares'] if schedule['total_shares'] > 0 else 0
        print(f"\n{symbol}:")
        print(f"  Start Date: {schedule['start_date']}")
        print(f"  Days Bought: {schedule['days_bought']}/{BUY_DAYS}")
        print(f"  Total Shares: {schedule['total_shares']:.0f}")
        print(f"  Total Cost: ${schedule['total_cost']:,.2f}")
        print(f"  Average Cost: ${avg_cost:.2f}")
else:
    print("No active buy schedules.")


### Provide OTC Symbols Manually

Since Alpaca's OTC coverage may be limited, you can provide your own list of OTC symbols to screen:


In [ ]:
# Example: Provide a list of OTC symbols to screen
# Replace with actual OTC symbols available on Alpaca
OTC_SYMBOLS_TO_SCREEN = [
    # 'SYMBOL1',
    # 'SYMBOL2',
    # 'SYMBOL3',
    # Add your OTC symbols here
]

# Run with custom symbol list
if OTC_SYMBOLS_TO_SCREEN:
    daily_screen_and_trade(OTC_SYMBOLS_TO_SCREEN)
else:
    print("Please add OTC symbols to the OTC_SYMBOLS_TO_SCREEN list above.")
